# [CSC8101] Big Data Analytics - 2022 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
        #print("Number of rows with task 1.2: {t:,}".format(t = _trips_12.count()))
    else:
        _trips_12 = _trips_11
        #print("Number of rows without task 1.2: {t:,}".format(t = _trips_12.count()))

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)
    
    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'delta'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

Trips dataset loaded!
 ---
 Size: S
 Format: delta
 Tables loaded: /FileStore/tables/taxi/taxi-S-delta/
 Number of trips (dataset rows): 2,898,033

In [0]:
print_count(trips)

Row count: 2,898,033

In [0]:
# dataset schemas
trips.printSchema()

root
-- index: long (nullable = true)
-- VendorID: double (nullable = true)
-- tpep_pickup_datetime: string (nullable = true)
-- tpep_dropoff_datetime: string (nullable = true)
-- passenger_count: double (nullable = true)
-- trip_distance: double (nullable = true)
-- RatecodeID: double (nullable = true)
-- store_and_fwd_flag: string (nullable = true)
-- PULocationID: long (nullable = true)
-- DOLocationID: long (nullable = true)
-- payment_type: double (nullable = true)
-- fare_amount: double (nullable = true)
-- extra: double (nullable = true)
-- mta_tax: double (nullable = true)
-- tip_amount: double (nullable = true)
-- tolls_amount: double (nullable = true)
-- improvement_surcharge: double (nullable = true)
-- total_amount: double (nullable = true)
-- congestion_surcharge: double (nullable = true)
-- cab_type: string (nullable = true)
-- lpep_pickup_datetime: string (nullable = true)
-- lpep_dropoff_datetime: string (nullable = true)
-- ehail_fee: double (nullable = true)
-- trip_type: double (nullable = true)

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
-- LocationID: long (nullable = true)
-- Borough: string (nullable = true)
-- Zone: string (nullable = true)
-- service_zone: string (nullable = true)

In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.438 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
#print_count(trips.filter((trips.trip_distance>0) | ((trips.passenger_count>0) & (trips.total_amount>0))))

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips dataset
    # removing rows where either (df.trip_distance==0) or ((df.passenger_count==0) and (df.total_amount==0)
    df = df.filter(~((df.trip_distance==0) | ((df.passenger_count==0) & (df.total_amount==0))))
    # selecting only required 5 columns
    df = df.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')
    return df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
display(trips_11.take(10))

Row count: 2,858,164

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3
114,90,1.6,1.0,12.8
90,144,1.8,1.0,13.3
114,48,2.0,1.0,14.75
48,152,5.7,1.0,22.3
234,148,1.8,1.0,14.75


In [0]:
#Avoid using trips_11.approxQuantile("total_amount", [0.5], 0.00) due to slow execution

In [0]:
#trips_11.select(ps.functions.percentile_approx("total_amount", 0.5, 1000000).alias("median")).collect()[0][0]
#ps.functions.percentile_approx(trips_11.total_amount, [0.5], 1000000)

In [0]:
# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    # calculating median value of total amount
    total_amt_med = df.select(ps.functions.percentile_approx("total_amount", 0.5)).collect()[0][0]
    # calculating median value of trip distances
    trip_dist_med = df.select(ps.functions.percentile_approx("trip_distance", 0.5)).collect()[0][0]
    
    # creating column with z scores for total amount
    df = df.withColumn("total_amount_z", ps.functions.abs((df.total_amount - total_amt_med) /
                                                          (1.438 * df.select(ps.functions.percentile_approx(ps.functions.abs(df.total_amount - total_amt_med), 0.5)).collect()[0][0])))
    # creating column with z scores for trip distances
    df = df.withColumn("trip_distance_z", ps.functions.abs((df.trip_distance - trip_dist_med) /
                                                           (1.438 * df.select(ps.functions.percentile_approx(ps.functions.abs(df.trip_distance - trip_dist_med), 0.5)).collect()[0][0])))
    
    # removing outliers
    df = df.filter(~((ps.functions.abs(df.total_amount_z)>3.5) | (ps.functions.abs(df.trip_distance_z)>3.5)))
    
    # removing z scire column
    df = df.drop("total_amount_z", "trip_distance_z")
    #df.printSchema()
    #df.show(10)
    
    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
#display(trips_12.take(10))

Row count: 2,483,577

## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [0]:
zone_names.printSchema()

root
-- LocationID: long (nullable = true)
-- Borough: string (nullable = true)
-- Zone: string (nullable = true)
-- service_zone: string (nullable = true)

In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    #adding pick-up zone names
    df = df.join(zones_df, df.PULocationID == zones_df.LocationID).drop("LocationID", "Borough", "service_zone")
    df = df.withColumnRenamed("Zone", "PUzone")
    
    #adding drop-off zone names
    df = df.join(zones_df, df.DOLocationID == zones_df.LocationID).drop("LocationID", "Borough", "service_zone")
    df = df.withColumnRenamed("Zone", "DOzone")
    #df.show(10)
    
    return df

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)
trips_21.printSchema()
print_count(trips_21)
# display(trips_21.take(10))

root
-- PULocationID: long (nullable = true)
-- DOLocationID: long (nullable = true)
-- trip_distance: double (nullable = true)
-- passenger_count: double (nullable = true)
-- total_amount: double (nullable = true)
-- PUzone: string (nullable = true)
-- DOzone: string (nullable = true)

Row count: 2,483,577

In [0]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    # input: output of task 2.1
    # calculating unit profitability
    df = df.withColumn("unit_profitabilty", df.total_amount / df.trip_distance)
    #df.show()
    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)
trips_22.printSchema()
print_count(trips_22)
# display(trips_22.take(10))

root
-- PULocationID: long (nullable = true)
-- DOLocationID: long (nullable = true)
-- trip_distance: double (nullable = true)
-- passenger_count: double (nullable = true)
-- total_amount: double (nullable = true)
-- PUzone: string (nullable = true)
-- DOzone: string (nullable = true)
-- unit_profitabilty: double (nullable = true)

Row count: 2,483,577

## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below
'''
type(
trips_22.groupBy(['PULocationID', 'DOLocationID'])\
                .agg(ps.functions.avg('unit_profitabilty').alias('average_unit_profitability'),\
                     ps.functions.count('unit_profitabilty').alias('trips_count'),\
                     ps.functions.sum('unit_profitabilty').alias('total passengers')
                    )
)
'''

Out[21]: "\ntype(\ntrips_22.groupBy(['PULocationID', 'DOLocationID']) .agg(ps.functions.avg('unit_profitabilty').alias('average_unit_profitability'), ps.functions.count('unit_profitabilty').alias('trips_count'), ps.functions.sum('unit_profitabilty').alias('total passengers')\n )\n)\n"

In [0]:
## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2
    # grouping the data based on 'PULocationID' and 'DOLocationID' followed by computing averages for each zone pair
    df = df.groupBy(['PULocationID', 'DOLocationID'])\
            .agg(ps.functions.avg('unit_profitabilty').alias('average_unit_profitability'),\
                 ps.functions.count('unit_profitabilty').alias('trips_count'),\
                 ps.functions.sum('passenger_count').alias('total_passengers')
                )
    #df.show(10)
    return df

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
#display(graph.take(10))

Row count: 13,299

In [0]:
'''
display(
    zone_names.join(graph, graph.PULocationID == zone_names.LocationID).drop("LocationID", "Borough", "service_zone") \
            .groupby('zone')\
            .agg(ps.functions.sum('trips_count').alias('zone_trips_count'),\
                 ps.functions.avg('average_unit_profitability').alias('average_zone_profitability'),\
                 ps.functions.sum('total_passengers').alias('zone_total_passengers')
                ).sort(ps.functions.col('zone_trips_count').desc()).take(10)
        )
'''

Out[24]: '\ndisplay(\n zone_names.join(graph, graph.PULocationID == zone_names.LocationID).drop("LocationID", "Borough", "service_zone") .groupby(\'zone\') .agg(ps.functions.sum(\'trips_count\').alias(\'zone_trips_count\'), ps.functions.avg(\'average_unit_profitability\').alias(\'average_zone_profitability\'), ps.functions.sum(\'total_passengers\').alias(\'zone_total_passengers\')\n ).sort(ps.functions.col(\'zone_trips_count\').desc()).take(10)\n )\n'

In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    # summarizing the zone pair informarion
    # getting the value of each of trips from any zone based on 'PULocationID' and computing summary statistics
    df = df.groupby('PULocationID')\
            .agg(ps.functions.sum('trips_count').alias('zone_trips_count'),\
                 ps.functions.mean('average_unit_profitability').alias('average_zone_profitability'),\
                 ps.functions.sum('total_passengers').alias('zone_total_passengers')
                )
    # adding zone names to the summary data
    df = zones_df.join(df, df.PULocationID == zones_df.LocationID).drop("LocationID", "Borough", "service_zone")
    # using collect style call to execute all the code previously and improving the execution time
    df = spark.createDataFrame(df.collect())
    
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    #taking top 10 trips zones
    df_zones = df_zones.sort(ps.functions.col('zone_trips_count').desc()).drop('average_zone_profitability', 'zone_total_passengers', 'PULocationID').head(10)
    df_zones = spark.createDataFrame(df_zones)
    #display(df_zones)
    #df_zones.show()
    return df_zones

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    #taking top 10 profit zones
    df_zones = df_zones.sort(ps.functions.col('average_zone_profitability').desc()).drop('zone_trips_count', 'zone_total_passengers', 'PULocationID').head(10)
    df_zones = spark.createDataFrame(df_zones)
    #display(df_zones)
    #df_zones.show()
    return df_zones

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    #taking top 10 passengers zones
    df_zones = df_zones.sort(ps.functions.col('zone_total_passengers').desc()).drop('zone_trips_count', 'average_zone_profitability', 'PULocationID').head(10)
    df_zones = spark.createDataFrame(df_zones)
    #display(df_zones)
    #df_zones.show()
    return df_zones

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
top10_trips.show()

+--------------------+----------------+
 Zone|zone_trips_count|
+--------------------+----------------+
Upper East Side S...| 122096|
Upper East Side N...| 103951|
Penn Station/Madi...| 96933|
 Midtown Center| 93776|
 Murray Hill| 88617|
 Midtown East| 85316|
 Lincoln Square East| 81103|
Upper West Side S...| 74910|
 Clinton East| 74673|
 Lenox Hill West| 74439|
+--------------------+----------------+

In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
top10_profit.show()

+--------------------+--------------------------+
 Zone|average_zone_profitability|
+--------------------+--------------------------+
Bay Terrace/Fort ...| 311.9330683658483|
 Baisley Park| 100.00592710440851|
 Prospect Park| 93.05468065107412|
Saint Michaels Ce...| 74.82831383792275|
Van Cortlandt Vil...| 48.74415903037328|
 Grymes Hill/Clifton| 38.65|
 Crotona Park| 36.84437137879827|
Flatbush/Ditmas Park| 33.88362306081024|
 Eastchester| 32.08567975083938|
Flushing Meadows-...| 29.564931153882387|
+--------------------+--------------------------+

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
top10_passenger.show()

+--------------------+---------------------+
 Zone|zone_total_passengers|
+--------------------+---------------------+
Upper East Side S...| 170285.0|
Upper East Side N...| 144281.0|
Penn Station/Madi...| 137494.0|
 Midtown Center| 135150.0|
 Murray Hill| 124088.0|
 Midtown East| 120019.0|
 Lincoln Square East| 114953.0|
 Clinton East| 107395.0|
Upper West Side S...| 106149.0|
 East Village| 105818.0|
+--------------------+---------------------+

## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'parquet'
WITH_TASK_12 = True

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

Trips dataset loaded!
 ---
 Size: XXL
 Format: parquet
 Tables loaded: ['/FileStore/tables/taxi/tripdata_1_6_2019.parquet', '/FileStore/tables/taxi/tripdata_7_12_2019.parquet', '/FileStore/tables/taxi/tripdata_2020_21.parquet']
 Number of trips (dataset rows): 132,396,785

In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

Out[31]: [DataFrame[Zone: string, zone_trips_count: bigint],
 DataFrame[Zone: string, average_zone_profitability: double],
 DataFrame[Zone: string, zone_total_passengers: double]]

S:  2.898033

  M:  15.571166

  L:  41.953716

 XL:  90.443069

XXL:  132.396785

_Table 1.0 Pipeline performance for `delta` format with collect in t32_summarise_zones_pairs._

| metric                      | S           | M           | L           | XL         | XXL       |
|-----------------------------|-------------|-------------|-------------|------------|-----------|
| rows (M)                    | 2.89        | 15.57       | 41.95       | 90.44      | 132.4     |
| execution time     (w/o 1.2)| 2.73(s)     | 3.20(s)     | 4.20(s)     | 6.51(s)    | 8.25(s)   |
| execution time              | 8.44(s)     | 12.51(s)    | 16.34(s)    | 35.58(s)   | 46.84(s)  |
| sec / 1M records (w/o 1.2)  | 0.78(s)     | 0.20(s)     | 0.10(s)     | 0.07(s)    | 0.06(s)   |
| sec / 1M records            | 2.92(s)     | 0.80(s)     | 0.38(s)     | 0.39(s)    | 0.35(s)   |


_Table 2. Pipeline performance for `delta` format without collect in t32_summarise_zones_pairs._

| metric                      | S           | M           | L           | XL         | XXL       |
|-----------------------------|-------------|-------------|-------------|------------|-----------|
| rows (M)   (w/o 1.2)        |2.827824     | 15.57       | 41.95       | 90.44      | 132.4     |
| rows (M)                    | 2.898       | 15.57       | 41.95       | 90.44      | 132.4     |
| execution time     (w/o 1.2)| 6.56(s)     | 8.16(s)     | 8.77(s)     | 17.10(s)   | 22.93(s)  |
| execution time              |14.23(s)     | 29.82(s)    | 52.77(s)    | 2.26(m)    | 2.33(m)   |
| sec / 1M records (w/o 1.2)  | 2.26(s)     | 0.52(s)     | 0.21(s)     | 0.19(s)    | 0.17(s)   |
| sec / 1M records            | 4.91(s)     | 1.92(s)     | 1.26(s)     | 1.5(s)     | 1.46(s)   |

_Table 1. Pipeline performance for `parquet` format with collect in t32_summarise_zones_pairs._

| metric                      | S           | M           | L           | XL         | XXL       |
|-----------------------------|-------------|-------------|-------------|------------|-----------|
| rows (M)                    | 2.89        | 15.57       | 41.95       | 90.44      | 132.4     |
| execution time   (w/o 1.2)  | 3.19(s)     | 39.46(s)    | 6.42(m)     | 8.51(m)    | 8.62(m)   |
| execution time              | 11.06(s)    | 2.70(m)     | 23.58(m)    | 32.00(m)   | 34.84(m)  |
| sec / 1M records (w/o 1.2)  | 1.10(s)     | 2.53(s)     | 9.18(s)     | 5.65(s)    | 3.90(s)   |
| sec / 1M records            | 3.82(s)     | 10.40(s)    | 33.72(s)    | 21.22(s)   | 15.78(s)  |


_Table 1. Pipeline performance for `parquet` format without collect in t32_summarise_zones_pairs._

| metric                      | S           | M           | L           | XL         | XXL       |
|-----------------------------|-------------|-------------|-------------|------------|-----------|
| rows (M)                    | 2.89        | 15.57       | 41.95       | 90.44      | 132.4     |
| execution time   (w/o 1.2)  | 8.06(s)     | 1.85(m)     |17.79(m)     | 24.37(m)   | 25.29(m)  |
| execution time              | 15.18(s)    | 3.73(m)     | 29.48(m)    | 40.99(m)   |41.03(m)  |
| sec / 1M records (w/o 1.2)  | 2.78(s)     | 7.13(s)     | 25.44(s)    | 16.17(s)   | 11.46(s)  |
| sec / 1M records            | 5.24(s)     | 14.37(s)    | 42.15(s)    | 27.19(s)   | 18.59(s)  |

# Analysis

From the execution times above we concurr that azure spark is optimized for delta data format and the use of collect style call at t32_summary significantly improves the executin times.